In [1]:
from glob import glob
from os import path
import seaborn as sns
import json
import os
import numpy as np
import pandas as pd
import re
from os.path import basename, splitext
import time
import pickle

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval_w_baseline

/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
city = 'berlin'

# fixed params
RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

refresh = False

In [3]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()

#Subset to frankfurt
list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find(city) > -1):
        list_city_dt_subset.append(city_db)

print(list_city_dt_subset)

['berlin_db=0.50', 'berlin_db=1.00', 'berlin_db=2.00', 'berlin_db=4.00']


In [7]:
for base_file in list_city_dt_subset:

    #create inputs to run a trial workforce size
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"

    expand_workforce_to_regions=True
    workforce_dict = {}

    # for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    for OC in [1.2]:
        # for model in ['fixed', 'flex', 'partflex']:
        for model in ['fixed']:
            if model == 'partflex':
                for max_n_shifts in range(2,5):

                    results_file = f"../raw_results/{base_file}_OC={OC}_model={model}_max_n_shifts={max_n_shifts}.pkl"
                    # if not os.path.exists(results_file) or refresh:

                    start_time = time.time()
                    dict_baseline, dict_results = \
                        run_roster_solver_objval_w_baseline(model, instance_file_weekday, 
                                                            shift_file_weekday, instance_file_weekend, 
                                                            shift_file_weekend, workforce_dict, 
                                                            OC, RM, GM, h_min, h_max, max_n_diff, 
                                                            max_n_shifts, expand_workforce_to_regions=True)
                    
                    dict_results['baseline'] = dict_baseline
                    end_time = time.time()
                    run_time = (end_time - start_time)
                    
                    # Save results
                    with open(results_file, 'wb') as file:
                        pickle.dump(dict_results, file)
                    print(f'Base: {base_file}, OC: {OC}, model: {model}-{max_n_shifts}, run_time: {run_time}')
            else:
                
                results_file = f"../raw_results/{base_file}_OC={OC}_model={model}.pkl"
                # if not os.path.exists(results_file) or refresh:
                start_time = time.time()
                dict_baseline, dict_results = \
                    run_roster_solver_objval_w_baseline(model, instance_file_weekday, 
                                                        shift_file_weekday, instance_file_weekend, 
                                                        shift_file_weekend, workforce_dict, 
                                                        OC, RM, GM, h_min, h_max, max_n_diff, 
                                                        expand_workforce_to_regions=True)
                
                dict_results['baseline'] = dict_baseline
                end_time = time.time()
                run_time = (end_time - start_time)

                # Save results
                with open(results_file, 'wb') as file:
                    pickle.dump(dict_results, file)
                print(f'Base: {base_file}, OC: {OC}, model: {model}, run_time: {run_time}')


Base: berlin_db=0.50, OC: 1.2, model: fixed, run_time: 22.508628845214844
Base: berlin_db=1.00, OC: 1.2, model: fixed, run_time: 32.71034622192383


KeyboardInterrupt: 

In [6]:
dict_results

{'regions': [3, 1, 0, 2],
 'reg_areas': {3: ['10789',
   '10777',
   '10779',
   '10717',
   '10719',
   '10707',
   '10585',
   '10589',
   '10625',
   '10587',
   '10623',
   '10627',
   '10629',
   '10559',
   '10555',
   '10553',
   '10551',
   '10713',
   '10711',
   '10709',
   '10825',
   '10715'],
  1: ['10409',
   '10557',
   '10117',
   '10119',
   '10115',
   '10179',
   '10178',
   '10435',
   '10439',
   '10437',
   '10405',
   '10407'],
  0: ['10319',
   '10318',
   '10245',
   '10249',
   '10247',
   '10369',
   '10367',
   '10365',
   '10317',
   '10315'],
  2: ['10787',
   '10781',
   '10783',
   '10785',
   '10963',
   '10997',
   '10969',
   '10965',
   '10961',
   '10967',
   '10999',
   '10243',
   '10823',
   '10829',
   '10827']},
 'shifts': {(3, 0): [0, 4],
  (3, 1): [0, 4],
  (3, 2): [0, 4],
  (3, 3): [0, 4],
  (3, 4): [0, 4],
  (3, 5): [0, 4],
  (3, 6): [0, 4],
  (1, 0): [0, 4],
  (1, 1): [0, 4],
  (1, 2): [0, 4],
  (1, 3): [0, 4],
  (1, 4): [0, 4],
  (1, 5): 